In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from scipy.stats import skew, kurtosis
from scipy.integrate import simpson
import pickle
import pandas as pd
from scipy.interpolate import RBFInterpolator

## Load and combine raw data from all three batches

In [5]:
batch1 = pickle.load(open(r'../Data/batch1.pkl', 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

In [6]:
numBat1 = len(batch1.keys())
numBat1

41

In [9]:
batch2 = pickle.load(open(r'../Data/batch2.pkl','rb'))

In [10]:
# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];

In [11]:
for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

In [12]:
del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

In [13]:
numBat2 = len(batch2.keys())
numBat2

43

In [14]:
batch3 = pickle.load(open(r'../Data/batch3.pkl','rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c42']
del batch3['b3c43']

In [15]:
numBat3 = len(batch3.keys())
numBat3

40

In [16]:
numBat = numBat1 + numBat2 + numBat3
numBat

124

In [17]:
bat_dict = {**batch1, **batch2, **batch3}

## Build dictionary 

In [19]:
feature_dict = {}
for id in bat_dict.keys():
    if id[1] in ["2", "3"]: feature_dict[id] = {"cycle_life": bat_dict[id]["cycle_life"][0][0]}
    else: feature_dict[id] = {"cycle_life": bat_dict[id]["cycle_life"]}

In [20]:
def crop_data(id,cycle):
    # isolate discharging data
    cropped = np.array([[Q, V] for Q, V, I in zip(bat_dict[id]["cycles"][cycle]["Qd"], bat_dict[id]["cycles"][cycle]["V"], bat_dict[id]["cycles"][cycle]["I"]) if I < -0.001])
    for i, datapoint in enumerate(cropped):
        if datapoint[1] < cropped[i+1, 1]:
            if datapoint[1] < 2.2: # identify where discharging ends
                end = i+1
                break
            # else: # this is an anomalous increase in voltage; remove so voltage is decreasing
            #     cropped[i+1, 1] = cropped[i, 1]
    cropped = cropped[:end]    
    x = np.flip(cropped[:, 1])
    y = np.flip(cropped[:, 0])
    return x,y

def get_Qd_resampled(id, cycle):
    x_list, y_list = crop_data(id, cycle)
    interpolated = RBFInterpolator(x_list[::3, np.newaxis], y_list[::3], degree=1)
    x_new = np.linspace(2.0, 3.59, 1000)[:, np.newaxis]
    return interpolated(x_new)

In [21]:
def get_var(id):
    diff = get_Qd_resampled(id, "100") - get_Qd_resampled(id, "10")
    return np.log10(np.var(diff))

def get_min(id):
    diff = get_Qd_resampled(id, "100") - get_Qd_resampled(id, "10")
    return np.log10(np.abs(np.min(diff)))

def get_mean(id):
    diff = get_Qd_resampled(id, "100") - get_Qd_resampled(id, "10")
    return np.log10(np.abs(np.mean(diff)))

def get_skew(id):
    diff = get_Qd_resampled(id, "100") - get_Qd_resampled(id, "10")
    return np.log10(np.abs(skew(diff)))

def get_kurt(id):
    diff = get_Qd_resampled(id, "100") - get_Qd_resampled(id, "10")
    return np.log10(np.abs(kurtosis(diff)))

In [22]:
for id in bat_dict.keys():
    feature_dict[id]["DeltaQ_logVar"] = get_var(id)
    feature_dict[id]["DeltaQ_logMin"] = get_min(id)
    feature_dict[id]["DeltaQ_logMean"] = get_min(id)
    feature_dict[id]["DeltaQ_logSkew"] = get_skew(id)
    feature_dict[id]["DeltaQ_logKurt"] = get_kurt(id)

## Discharge capacity curve features

In [23]:
def get_max(series):
    '''filter for outliers in max discharge capacity'''
    max_idx = np.argmax(series)
    while series[max_idx] > 1.3:
        series = np.delete(series,max_idx)
        max_idx = np.argmax(series)
    return series[max_idx]

In [24]:
for id in bat_dict.keys():
    # difference between max discharge capacity and cycle 2
    #feature_dict[id]["QD_Max-2"] = (np.max(bat_dict[id]["summary"]["QD"])-bat_dict[id]["summary"]["QD"][1])
    feature_dict[id]["QD_Max-2"] = (get_max(bat_dict[id]["summary"]["QD"])-bat_dict[id]["summary"]["QD"][1])

    # discharge capacity at cycle 2
    feature_dict[id]["QD_2"] = bat_dict[id]["summary"]["QD"][1]

    # slope of linear fit to capacity fade curve, cycles 2 to 100
    linear_fit = np.polyfit(np.linspace(2,100,99),bat_dict[id]["summary"]["QD"][1:100],deg=1)
    feature_dict[id]["slope_capacity_fade_2-100"] = linear_fit[0]

    # intercept of linear fit to capacity fade curve, cycles 2 to 100
    feature_dict[id]["intercept_capacity_fade_2-100"] = linear_fit[1]

    # slope of linear fit to capacity fade curve, cycles 91 to 100
    linear_fit_2 = np.polyfit(np.linspace(91,100,10),bat_dict[id]["summary"]["QD"][90:100],deg=1)
    feature_dict[id]["slope_capacity_fade_91-100"] = linear_fit_2[0]

    # intercept of linear fit to capacity fade curve, cycles 91 to 100
    feature_dict[id]["intercept_capacity_91-100"] = linear_fit_2[1]


## Other Features

In [25]:
for id in bat_dict.keys():
    # average charge time, first five cycles
    feature_dict[id]["init_avg_charge_time"] = np.mean(bat_dict[id]["summary"]["chargetime"][1:6])

    # average temperature over time, cycles 2 through 100
    #feature_dict[id]["avg_T"] = np.mean(bat_dict[id]["summary"]["Tavg"][1:100])

    # integral of temperature, cycles 2 through 100
    feature_dict[id]["T_integral"] = simpson(bat_dict[id]["summary"]["Tavg"][1:100],np.linspace(1,99,99))

    # minimum internal resistance, cycles 2 through 100
    feature_dict[id]["min_IR"] = np.min(bat_dict[id]["summary"]["IR"][1:100])

    # internal resistance, difference between cycle 100 and cycle 2
    feature_dict[id]["IR_100-2"] = bat_dict[id]["summary"]["IR"][99]-bat_dict[id]["summary"]["IR"][1]

In [32]:
feature_df = pd.DataFrame.from_dict(feature_dict,orient="index")

# feature_df = feature_df.drop(feature_df.columns[0], 1)
feature_df.head()

,cycle_life,DeltaQ_logVar,DeltaQ_logMin,DeltaQ_logMean,DeltaQ_logSkew,DeltaQ_logKurt,QD_Max-2,QD_2,slope_capacity_fade_2-100,intercept_capacity_fade_2-100,slope_capacity_fade_91-100,intercept_capacity_91-100,init_avg_charge_time,T_integral,min_IR,IR_100-2
b1c0,1852.0,-5.038748,-2.052228,-2.052228,-0.132553,-0.067138,0.006393,1.070689,-0.000207,1.091144,0.000035,1.072425,13.374894,3097.354687,0.016444,-0.000075
b1c1,2160.0,-5.128559,-2.062196,-2.062196,-0.467106,0.014732,0.009288,1.075301,0.000006,1.080965,-0.000041,1.084767,13.409150,3070.296078,0.000000,-0.000042
b1c2,2237.0,-4.940384,-1.987750,-1.987750,-0.360252,0.024002,0.008131,1.079922,0.000010,1.084846,-0.000044,1.089364,13.358242,3084.836680,0.000000,-0.000014
b1c3,1434.0,-4.366865,-1.703802,-1.703802,-0.284044,0.022955,0.006530,1.079723,0.000017,1.084075,-0.000004,1.085131,12.025140,2934.438842,0.000000,0.000039
b1c4,1709.0,-4.587381,-1.840347,-1.840347,-0.345178,0.117097,0.005900,1.078417,0.000019,1.081875,-0.000029,1.085538,12.041851,3082.307734,0.000000,-0.000053


In [33]:
feature_df.to_csv("../Data/features5.csv", index=False)

## To read data

In [161]:
loaded_df = pd.read_csv("../Data/features.csv")
loaded_df.head()

,Unnamed: 0,cycle_life,DeltaQ_logVar,DeltaQ_Min,DeltaQ_Skew,DeltaQ_Kurt,QD_Max-2,QD_2,slope_capacity_fade_2-100,intercept_capacity_fade_2-100,slope_capacity_fade_91-100,intercept_capacity_91-100,init_avg_charge_time,avg_T,min_IR,IR_100-2
0,b1c0,1852.0,-4.836170,-0.009054,0.443416,0.342752,0.468365,1.070689,-0.000207,1.091144,0.000035,1.072425,13.374894,31.603747,0.016444,-0.000075
1,b1c1,2160.0,-4.988991,-0.010529,0.072319,-0.156308,0.009288,1.075301,0.000006,1.080965,-0.000041,1.084767,13.409150,31.330314,0.000000,-0.000042
2,b1c2,2237.0,-4.812421,-0.013186,0.059164,0.621248,0.008131,1.079922,0.000010,1.084846,-0.000044,1.089364,13.358242,31.479584,0.000000,-0.000014
3,b1c3,1434.0,-4.371544,-0.018933,-0.353850,-1.072029,0.006530,1.079723,0.000017,1.084075,-0.000004,1.085131,12.025140,29.942199,0.000000,0.000039
4,b1c4,1709.0,-4.533900,-0.017832,-0.257654,-0.549822,0.005900,1.078417,0.000019,1.081875,-0.000029,1.085538,12.041851,31.448884,0.000000,-0.000053
